In [1]:
import os
import uuid
import json
from tqdm.notebook import tqdm
import pandas as pd
from dotenv import load_dotenv
from typing import Type, List

from app.database.models import async_main, drop_all_tables, async_session
from app.database.requests import get_student, get_all, set_litwork
from app.llm.LLMClient import generate_questionary, generate_idea, discuss_litwork
from app.utils import question_to_text, has_litwork
import app.keyboards as kb
load_dotenv()

from app.database.models import Base, Student, Litwork, Topic

import warnings
warnings.filterwarnings('ignore')


### Добавляем данные в базу данных

In [2]:
load_dotenv()

TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
CATALOGUE_ID =  os.getenv("YC_CATALOGUE_ID")
IAM_TOKEN = os.getenv("IAM_TOKEN")
OAUTH_TOKEN = os.getenv("YANDEX_OAUTH_TOKEN")
user_text = "Сгенерируй задачу по математике с решением для ученика 11 класса"

async def set_dict_values(model: Type[Base], values):
    for val in values:
        await set_const_value(model, val)

async def apply_func_to_list(func, values, **kwargs):
    for val in values:
        await func(name=val, **kwargs)

def orm_list_to_df(orm_list: list | None) -> pd.DataFrame | None:
    if not orm_list:
        return None
    
    model_class = orm_list[0].__class__
    column_names = [column.name for column in model_class.__table__.columns]

    table = []
    for obj in orm_list:
        row = {col: getattr(obj, col) for col in column_names}
        table.append(row)

    return pd.DataFrame(table)

In [3]:
with open('app/database/literary_works.json', 'r') as f:
    litworks = json.load(f)

print(litworks)

[{'title': 'Blue Beard', 'author': 'Charles Perrault', 'path': 'data/blue_beard.txt'}, {'title': 'The Garden of Forking Paths', 'author': 'Jorge Luis Borges', 'path': 'data/borges_garden_fantastic.txt'}, {'title': 'The Theory of Adaptation', 'author': "Linda Hutcheon with Siobhan O'Flynn", 'path': 'data/theory_of_adaptation.txt'}, {'title': 'The Cycle of Socialization', 'author': 'Bobbie Harro', 'path': 'data/cycle_of_socialization.txt'}]


In [4]:
await drop_all_tables()
await async_main()

# Litworks:
for litwork in litworks:
    await set_litwork(litwork['title'], litwork['author'], litwork['path'])

All tables dropped successfully.


#### Смотрим на данные в базе

In [5]:
objects = await get_all(Litwork)
objects_df = orm_list_to_df(objects)
display(objects_df)

,id,title,author,path
0,1,Blue Beard,Charles Perrault,data/blue_beard.txt
1,2,The Garden of Forking Paths,Jorge Luis Borges,data/borges_garden_fantastic.txt
2,3,The Theory of Adaptation,Linda Hutcheon with Siobhan O'Flynn,data/theory_of_adaptation.txt
3,4,The Cycle of Socialization,Bobbie Harro,data/cycle_of_socialization.txt


In [6]:
# student = await get_student(738490613)
# student.topic_id, student.difficulty_id, student.task_type_id, student.exam_id